# Oak parameter data from TRY

In [19]:
import pandas as pd
import os
#import myfuncs
from datetime import datetime
pd.set_option('display.max_rows', 400)
pd.set_option('display.min_rows', 400)
import numpy as np

#define constants
m2_per_cm2 = 1e-4
g_biomass_per_g_C = 2
m2_per_mm2 = 1e-6
mg_per_g = 1e3
g_per_kg = 1000
mm2_per_cm2 = 100
g_per_mg = 1e-3
write_csv = True

In [5]:
path_to_trait_obs = "/home/adam/cloud/gdrive/postdoc/parameters/param_data/trait_observations"
write_QUKE_data_avail = False
write_to_csv = False

In [6]:
QUCH_try_data = os.path.join(path_to_trait_obs,"QUCH/TRY/TRY_obs_QUCH.txt")
QUKE_try_data = os.path.join(path_to_trait_obs,"QUKE/TRY/QUKE_TRYdata.txt")
CA_oak_try_data = os.path.join(path_to_trait_obs,"CA_oaks/TRY/CA_oak_try_data.txt")

### Functions

In [7]:
def convert_to_fates_units(trait_name,input_units,input_value):

    
    if ("SLA" in trait_name) | ("Specific leaf area" in trait_name) and input_units == "mm2 mg-1":  
        output_value = input_value * m2_per_mm2 * mg_per_g * g_biomass_per_g_C
        return output_value

    elif "Leaf nitrogen" in trait_name and input_units == "mg/g":
        output_value = input_value * g_per_mg * g_biomass_per_g_C
        return output_value

    elif "Stem specific density" in trait_name and input_units == "g/cm3":
        output_value = input_value
        return output_value
    
    # Anaker conversions
    elif trait_name == "log(SLA cm2 g-1)":
        output_value = np.exp(input_value) * m2_per_cm2 * g_biomass_per_g_C
        return output_value
        
    elif trait_name == 'log(wood density g ml-1)':
        output_value = np.exp(input_value)
        return output_value
        
    elif trait_name == 'log(N %)':
        output_value = np.exp(input_value) * 1e-2 * g_biomass_per_g_C
        return output_value
    
    #Jepson leaf N
    elif trait_name == "Leaf Nmass":
    
        # muliplying by 1e-2 is to reverse the "per cent"  
        output_value = input_value * 1e-2 * g_biomass_per_g_C
        return output_value
    
    elif "Wood density" in trait_name:
        output_value = input_value
        return output_value
    
    elif trait_name == "Leaf area to sapwood area" and input_units == "mm2 mm-2":
        output_value = input_value * 1e-4
        return output_value
    
    elif "Vcmax" in trait_name and input_units == "micro mol m-2 s-1":
        output_value = input_value
        return output_value
    
    else:
        #print("No unit conversion known for:", trait_name)
        return None
    
def convert_to_fates_param_name(input_trait_name):
    if ("SLA" in input_trait_name) | ('Specific leaf area' in input_trait_name):
        return "fates_leaf_slatop"
    
    elif ("wood density" in input_trait_name) |  ("Wood density" in input_trait_name):
        return "fates_wood_density"
    
    elif ("N %" in input_trait_name) | ("Leaf Nmass" in input_trait_name):
        return "fates_stoich_nitr"
    
    elif "Leaf nitrogen (N) content per leaf dry mass" in input_trait_name:
        return "fates_stoich_nitr"
    
    elif "Leaf area to sapwood area" in input_trait_name:
        return "fates_allom_la_per_sa_int"
    
    elif "Vcmax" in input_trait_name:
        return "fates_leaf_vcmax25top"
    
    else:
        return None
    
def get_sources(df,variable):
    return df.loc[df.fates_name == variable].source.unique()

def get_range_data(df,subset_name):

    summary_stats = df.groupby('fates_name')['fates_value'].agg(
        min_value='min',
        max_value='max',
        mean='mean',
        median='median',
        sample_size='size'
    )

    summary_stats['subset_name'] = subset_name
    
    return summary_stats

### View TRY data availability for black oak

Note: to view the data availability for QUKE look at the csv exported in this code block

In [8]:
#clean data
QUKE_data_avail = os.path.join(path_to_trait_obs,"QUKE/TRY/QUKE_TRY_obs_availability.txt")
QUKE_df = pd.read_csv(QUKE_data_avail, sep="\t", encoding='latin1')[["Trait","TraitID",".Quercus kelloggii."]]
QUKE_df = QUKE_df.rename(columns={"Trait": "trait", "TraitID": "trait_id", ".Quercus kelloggii.": "n"})
QUKE_df = QUKE_df.sort_values(["n"],ascending=False)

if write_QUKE_data_avail == True:
    QUKE_df.to_csv(os.path.join(path_to_trait_obs,"QUKE/TRY/QUKE_TRY_obs_availability_clean.csv"))

    #get trait IDs available for QUKE
    print(list(QUKE_df.trait_id))

    print(QUKE_df)

### Load, clean, and view oak trait data

In [9]:
# Load and join QUCH and QUKE data
try_df_raw_QUCH = pd.read_csv(QUCH_try_data, sep="\t", encoding='latin1')[['ObservationID','SpeciesName','AccSpeciesName','TraitID','TraitName','OrigValueStr','OrigUnitStr','StdValue','UnitName','ErrorRisk']]
try_df_raw_QUKE = pd.read_csv(QUKE_try_data, sep="\t", encoding='latin1')[['ObservationID','SpeciesName','AccSpeciesName','TraitID','TraitName','OrigValueStr','OrigUnitStr','StdValue','UnitName','ErrorRisk']]
try_df_raw_CA_oak = pd.read_csv(CA_oak_try_data, sep="\t", encoding='latin1')[['ObservationID','SpeciesName','AccSpeciesName','TraitID','TraitName','OrigValueStr','OrigUnitStr','StdValue','UnitName','ErrorRisk']]

try_df_raw = pd.concat([try_df_raw_QUCH,try_df_raw_QUKE,try_df_raw_CA_oak])

try_df_raw = try_df_raw[['AccSpeciesName','TraitName','OrigValueStr','OrigUnitStr','StdValue','UnitName']]

#print("Available trait observations")
#print(try_df_raw.TraitName.value_counts())
#print("\n")
#print(try_df_raw.info())

In [10]:
try_df_raw = pd.DataFrame.drop_duplicates(try_df_raw)

### Filter to relevant traits

In [11]:
# Filter to get traits of interest
fates_relevant_traits = ['Stem specific density (SSD, stem dry mass per stem fresh volume) or wood density',
                         'Leaf nitrogen (N) content per leaf dry mass',
                         'Leaf area per leaf dry mass (specific leaf area, SLA or 1/LMA): undefined if petiole is in- or excluded',
                         'Leaf area per leaf dry mass (specific leaf area, SLA or 1/LMA): petiole excluded',
                         'Photosynthesis carboxylation capacity (Vcmax) per leaf area (Farquhar model)']

try_df = try_df_raw[try_df_raw.TraitName.isin(fates_relevant_traits)]
try_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 368 entries, 16 to 53806
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   AccSpeciesName  368 non-null    object 
 1   TraitName       368 non-null    object 
 2   OrigValueStr    368 non-null    object 
 3   OrigUnitStr     368 non-null    object 
 4   StdValue        368 non-null    float64
 5   UnitName        368 non-null    object 
dtypes: float64(1), object(5)
memory usage: 20.1+ KB


In [12]:
#try_df.loc[try_df.TraitName == "Photosynthesis carboxylation capacity (Vcmax) per leaf area (Farquhar model)"]

### Apply unit conversions and fates parameter names

In [13]:
# Unit convertion to fates units
fates_values = try_df.apply(lambda x: convert_to_fates_units(x['TraitName'], x['UnitName'], x["StdValue"]), axis=1)
try_df['fates_value'] = fates_values

# Convert variable names to fates names
fates_names = try_df.apply(lambda x: convert_to_fates_param_name(x["TraitName"]),axis = 1)
try_df["fates_name"] = fates_names


/tmp/ipykernel_8277/4023397515.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  try_df['fates_value'] = fates_values
/tmp/ipykernel_8277/4023397515.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  try_df["fates_name"] = fates_names


### Get percentiles for oak vcmax

In [17]:
def calculate_percentiles(data, percentiles):
    """
    Calculate the specified percentiles of a dataset.

    :param data: list or ndarray, the dataset
    :param percentiles: list of floats, the percentiles to compute, e.g., [25, 50, 75]
    :return: dictionary of percentiles and their corresponding values
    """
    results = {}
    for p in percentiles:
        value = np.percentile(data, p)
        results[p] = value
    return results

In [14]:
try_df.loc[try_df["fates_name"] == "fates_leaf_vcmax25top"].describe()

,StdValue,fates_value
count,56.000000,56.000000
mean,65.111867,65.111867
std,28.269910,28.269910
min,13.630628,13.630628
25%,45.051047,45.051047
50%,64.207770,64.207770
75%,83.319989,83.319989
max,127.296133,127.296133


#### Low-end percentiles

In [23]:
calculate_percentiles(try_df.loc[try_df["fates_name"] == "fates_leaf_vcmax25top"]["fates_value"],[1,2,3,5,10])

{1: 14.906282625171244,
 2: 16.36395350384018,
 3: 18.64069777496117,
 5: 24.680039042699,
 10: 31.475380780524404}

### Get low-end percentiles for oak slatop

In [24]:
calculate_percentiles(try_df.loc[try_df["fates_name"] == "fates_leaf_slatop"]["fates_value"],[1,2,3,5,10])

{1: 0.008158152,
 2: 0.00991634399996065,
 3: 0.01021009999950812,
 5: 0.010993779999999998,
 10: 0.01179225806452}

### Organize and write csv

In [15]:
QUKE_df = try_df.loc[try_df.AccSpeciesName == "Quercus kelloggii"]
QUCH_df = try_df.loc[try_df.AccSpeciesName == "Quercus chrysolepis"]
QUDO_df = try_df.loc[try_df.AccSpeciesName == "Quercus douglasii"]

param_range_oak_df = pd.concat([get_range_data(QUKE_df,"QUKE"),get_range_data(QUCH_df,"QUCH"),get_range_data(QUDO_df,"QUDO")]).reset_index()
param_range_oak_df['sources'] = "try"
param_range_oak_df['pft'] = "oak"

In [16]:
# Write CSV
if write_to_csv == True:

    now = datetime.now()
    dt_string = now.strftime("%m-%d-%Y-%H-%M-%S")
    file_path = "oak_param_ranges_data_derived_" + dt_string + ".csv"
    param_range_oak_df.to_csv(file_path, index=False)